Preprocessing Data (Milestone 1)

In [1]:
import numpy as np

In [7]:
def clean(fname):

    # Separate file into books with all metadata
    startBook = "<book"
    books = []
    with open(fname) as openFile:
        for line in openFile:
            if line[:len(startBook)] == startBook:
                books.append(line)
            else:
                books[-1] += line
    openFile.close()

    books = np.array(books)

    # Collect synopses and all genre tags for each book
    # X_data found here (synopses)
    synopses = []
    genres = []

    startBody = "<body>"
    endBody = "</body>"

    startTopic = "<topics>"
    endTopic = "</topics>"

    for book in books:
        start = book.index(startBody) + len(startBody)
        end = book.index(endBody)
        synopses.append(book[start:end])

        start = book.index(startTopic) + len(startTopic)
        end = book.index(endTopic)
        genres.append(book[start:end])

    X_data = np.array(synopses)


    # Create matrix of genre tags to book
    # Y_data found here
    start = ">"
    end = "</"

    Y_data = np.zeros((len(books), len(allGenres)))

    for i in range(len(books[:200])):
        entry = genres[i]
        while entry.find(end) != -1:
            s = entry.index(start) + 1
            e = entry.index(end)
            genreTag = entry[s:e]
            Y_data[i][allGenres.index(genreTag)] = 1
            entry = entry[e+5:]

    return (X_data, Y_data)




In [8]:
# Create a list of all genre tags
allGenres = []
with open("../data/hierarchy.txt") as hierarchyFile:
    for line in hierarchyFile:
        tab = line.find("\t")
        g1 = line[:tab]
        g2 = line[tab+1:len(line)-1]
        if allGenres.count(g1) == 0:
            allGenres.append(g1)
        if allGenres.count(g2) == 0:
            allGenres.append(g2)

 # print(allGenres[:4])

In [102]:
X_train, Y_train = clean("../data/BlurbGenreCollection_EN_train.txt")

In [9]:
""" print(X_train[0])
print(Y_train[0])
print(np.where(Y_train[0]==1))
print(genres[0])
print(allGenres.index("Nonfiction"))
print(allGenres.index("Games"))  """

' print(X_train[0])\nprint(Y_train[0])\nprint(np.where(Y_train[0]==1))\nprint(genres[0])\nprint(allGenres.index("Nonfiction"))\nprint(allGenres.index("Games"))  '

In [12]:
X_test, Y_test = clean("../data/BlurbGenreCollection_EN_test.txt")
X_valid, Y_valid = clean("../data/BlurbGenreCollection_EN_dev.txt")

# print(X_test[0])
# print(Y_test[0])

# print(X_valid[0])
# print(Y_valid[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


Create the Model and Training! (Milestone 2)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.layers import Flatten, Conv1D, MaxPool1D
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy

In [16]:
model = Sequential()

# DEFINE seq_length as length of longest blurb...maybe?
nb_filters=16
filter_size=3
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal', input_shape=(seq_length, 1)))
model.add(MaxPool1D()) 
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(MaxPool1D()) 
model.add(Conv1D(filters=nb_filters, kernel_size=filter_size, activation='sigmoid', 
                 kernel_initializer='glorot_normal'))
model.add(MaxPool1D()) 
model.add(Flatten())
model.add(Dense(1, activation='linear'))

NameError: name 'seq_length' is not defined

In [18]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
es = EarlyStopping(monitor='val_loss', 
                   patience=50, verbose=1,
                   restore_best_weights=True)



In [ ]:
model.fit(X,Y, batch_size=32, validation_split=0.2, callbacks=[es], epochs=100)

In [21]:
preds = model.predict(X_test)

2021-11-18 21:26:10.570553: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
